In [ ]:
# Trigger the authentication flow.
ee.Authenticate()
# Initialize the library.
ee.Initialize()

In [ ]:
#AOI fyrir Vindefjallen study area
geojson_path = r"Data\Vindefjallen_data\outputs\shapefiles\Vindefjallen_extent_updated4326.json"

# Open the GeoJSON file
with open(geojson_path) as f:
    data = json.load(f)
aoi = ee.Geometry.Polygon(data['features'][0]['geometry']['coordinates'])

# get multitemporal data  (S1)
date_format = 'YYYY-MM-dd'
# date_format = 'dd/MM/YYYY'
date_1 = ee.Date('2017-01-01')
date_2 = ee.Date('2022-04-01')
im_coll = (ee.ImageCollection('COPERNICUS/S1_GRD')
           .filterBounds(aoi)
           .filterDate(date_1,date_2)
           .filter(ee.Filter.eq('instrumentMode', 'IW'))
           .map(lambda img: img.set('date', ee.Date(img.date()).format(date_format)))
           .sort('date'))

s1_coords_listi = im_coll.toList(im_coll.size()).getInfo()#converts to a ee list

s1_coords_listi.count('id')

In [ ]:
### GEE dictionary sem heldur utan um granule ID og hvort að það sé innan granule
#hver mæling úr excel skránni er testuð hvort að hún sé innan granule og búin að dictionary sem heldur utan um það
ordabok_granule_id_gee = {}

teljari = 0
for i in range(len(s1_coords_listi)):
    polygon = s1_coords_listi[i]['properties']['system:footprint']['coordinates']
    measurment_in_granule_ID = []
    measurment_in_granule_cornerTriangle = []
    measurment_in_granule_date = []
    measurment_in_granule_depth = []
    measurment_in_granule_temperature = []
    # print(polygon)
    for j in range(len(vindefjallen_triangleCorners_numpy_celcius)):
        point = (vindefjallen_triangleCorners_numpy_celcius[j][6], vindefjallen_triangleCorners_numpy_celcius[j][7]) #x,y coordinates from excel file
        
        if Point_in_polygon(point[0], point[1], polygon):
            #check if dates are the same, otherwise it will add to each granule where there is a match, but the dates are not the same
            if vindefjallen_triangleCorners_numpy_celcius[j][3] == s1_coords_listi[i]['properties']['date']:
                measurment_in_granule_cornerTriangle.append(vindefjallen_triangleCorners_numpy_celcius[j][5])
                measurment_in_granule_date.append(vindefjallen_triangleCorners_numpy_celcius[j][3])
                measurment_in_granule_depth.append(vindefjallen_triangleCorners_numpy_celcius[j][4])
                measurment_in_granule_ID.append(vindefjallen_triangleCorners_numpy_celcius[j][0])
                measurment_in_granule_temperature.append(vindefjallen_triangleCorners_numpy_celcius[j][8])
    if len(measurment_in_granule_date) != 0:
        #If the length of the list is not 0, then it will add the granule to the dictionary
        ordabok_granule_id_gee[teljari] = {
                                'granule_ID': s1_coords_listi[i]['properties']['system:index'], 
                                'orbitProp': s1_coords_listi[i]['properties']['orbitProperties_pass'],
                                'pathNr': s1_coords_listi[i]['properties']['relativeOrbitNumber_stop'],
                                'date': s1_coords_listi[i]['properties']['date'],
                                'measurement_date': measurment_in_granule_date[0],
                                'measurement_id': measurment_in_granule_ID,
                                'measurement_cornerTriangle': measurment_in_granule_cornerTriangle,
                                'measurement_depth': measurment_in_granule_depth,
                                'measurement_temperature': measurment_in_granule_temperature,
                                'nr_of_measurements': len(measurment_in_granule_ID),
                                'polygon': polygon
                                }
        teljari += 1
# ordabok_granule_id
print(len(ordabok_granule_id_gee))
xgee = ordabok_granule_id_gee[0].keys()
for i in xgee:
    print(i)

In [ ]:
#Study area outlines
aoi = 'POLYGON((15.5702 66.3423,15.4488 66.3354,15.4242 66.2903,15.346 66.2493,15.2874 66.2273,15.2277 66.2165,15.145 66.1704,15.1619 66.0775,15.2024 66.0619,15.3474 66.0313,15.5304 65.9813,15.6281 65.9312,15.63 65.8965,15.741 65.8534,15.8627 65.8514,16.0288 65.8348,16.1754 65.8159,16.2049 65.8012,16.339 65.7877,16.4512 65.8283,16.2466 65.864,16.121 65.9472,16.1321 66.0205,16.2592 66.0296,16.4784 65.9773,16.6739 65.9642,16.8497 65.9721,16.9572 66.0108,16.7932 66.0502,16.5999 66.0381,16.4575 66.0817,16.2843 66.1399,16.2622 66.1561,15.9537 66.251,15.5702 66.3423))'

opts_slc = {
    'platform': asf.PLATFORM.SENTINEL1,
    'beamMode': asf.BEAMMODE.IW,
    'start': '2017-01-01T01:00:00Z',
    'end': '2023-06-11T23:59:59Z',
    'processingLevel' : asf.PRODUCT_TYPE.SLC
}
asf_coords_slc = asf.geo_search(intersectsWith=aoi, **opts_slc)
print(f'{len(asf_coords_slc)} results found - SLC')

opts_grd = {
    'platform': asf.PLATFORM.SENTINEL1,
    'beamMode': asf.BEAMMODE.IW,
    'start': '2017-01-01T01:00:00Z',
    'end': '2023-06-11T23:59:59Z',
    'processingLevel' : asf.PRODUCT_TYPE.GRD_HD
}
asf_coords_grd = asf.geo_search(intersectsWith=aoi, **opts_grd)

print(f'{len(asf_coords_grd)} results found - GRD_HD')